In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.cluster import KMeans

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if neck ==0:
        return([0])
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0:
        torso = distance.euclidean(hip2, neck)
    else:
        torso = distance.euclidean(hip1, neck)
    return(torso)

In [ ]:
def find_min_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,14,15,16,17]])#remove the arms from the equation cause the bounding boxes are too large 
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,14,15,16,17]])
    x = round(np.max(array))
    return(x)

In [ ]:
path_folders = Path('/Users/andreibirladeanu/Documents/Data/to_track')
path_videos = Path('/Users/andreibirladeanu/Documents/Data/videos_clustering')

In [ ]:
folders = sorted_alpha(path_folders)
videos = sorted_alpha(path_videos)

In [ ]:
folders[:2]

In [ ]:
for folder, video in zip(folders, videos):
    print(folder)
    cap = cv2.VideoCapture(os.path.join(path_videos, video))
    samples = []
    skeletons = []
    for skeleton in sorted_alpha(os.path.join(path_folders, folder)):
        skeletons.append(skeleton)
        for coord in sorted_alpha(os.path.join(path_folders,folder,skeleton)):
            cap.set(1, int(coord.split(".")[0])-1)
            ret, image = cap.read()
            if ret == False:
                continue
            data = pd.read_csv(os.path.join(path_folders,folder,skeleton,coord))
            img = image[find_min_round(data['y']):find_max_round(data['y']),find_min_round(data['x']):find_max_round(data['x'])]
            if img.shape[0]==0 or img.shape[1]==0:
                samples.append({coord:[torso_size(data),0,0,0]})
                continue
            b,g,r = cv2.split(img)
            samples.append({coord:[torso_size(data),np.mean(b),np.mean(g), np.mean(r)]})

    X = np.array([list(x.values())[0] for x in samples])
    labels = KMeans(n_clusters=2, random_state=0).fit_predict(X)
    new_sk1 = Path(os.path.join(path_folders, folder, 'label_0'))
    new_sk2 = Path(os.path.join(path_folders, folder, 'label_1'))
    new_sk1.mkdir(parents=True,exist_ok=True); new_sk2.mkdir(parents=True,exist_ok=True)
    first_skel = set(sorted_alpha(os.path.join(path_folders, folder, skeletons[0])))
    second_skel = set(sorted_alpha(os.path.join(path_folders, folder, skeletons[1])))
    count = 0
    for label, sample in zip(labels, samples):
        file = list(sample.keys())[0]
        count += 1
        if count <= len(first_skel):
            if file in first_skel:
                if label == 0:
                    os.rename(os.path.join(path_folders,folder,skeletons[0], file) ,os.path.join(new_sk1, file))
                else:
                    os.rename(os.path.join(path_folders,folder,skeletons[0], file) ,os.path.join(new_sk2, file))
        else:
            if file in second_skel:
                if label == 0:
                    os.rename(os.path.join(path_folders,folder,skeletons[1], file) ,os.path.join(new_sk1, file))
                else:
                    os.rename(os.path.join(path_folders,folder,skeletons[1], file) ,os.path.join(new_sk2, file))
    shutil.rmtree(os.path.join(path_folders, folder,skeletons[0]))
    shutil.rmtree(os.path.join(path_folders, folder,skeletons[1]))
        
print('success')       

In [ ]:
print('success')

In [ ]:
a= set([2,4,5])
len(a)

In [ ]:
len(samples)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
for x, sample in zip(X, samples):
    print(kmeans.predict([x])[0])

In [ ]:
list(samples[0].keys())[0].parent

In [ ]:
import subprocess as sbp
import os

path='/Users/andreibirladeanu/Documents/Data/meal_tracked_clustering/1053_meal/skel1'
fol = os.listdir(path)
p2 = '/Users/andreibirladeanu/Documents/Data/meal_tracked_clustering/1053_meal/skel3'

for i in fol:
    p1 = os.path.join(path,i)
    p3 = 'cp -r ' + p1 +' ' + p2+'/.'
    sbp.Popen(p3,shell=True)